# Pruebas MultiFADL One-versus-Rest. hepatitis dataset

In [6]:
import pickle
import json

import pandas as pd
import tensorflow as tf
import numpy as np
from src import utils

from src.MonoFADLModel import MonoFADLModel
from src.MultiFADLModelOvR import MultiFADLModelOvR
from src.NoSelectionModel import NoSelectionModel

# Seed for neural network executions
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [7]:
comparative_results = pd.DataFrame(columns=['test_loss', 'test_accuracy', 'test_f1', 'number of selected features', 'selected Features'])

* Preprocesamiento

In [8]:
# Load dataset and preprocess it
# https://www.kaggle.com/datasets/brunogrisci/hepatitis-cancer-gene-expression-cumida

# Check if stored, otherwise load and store
try:
    hepatitis = pickle.load(open('data/hepatitis_egypt.pkl', 'rb'))
except:
    hepatitis = pd.read_csv('data/hepatitis_egypt.csv')
    
    # Renombrar sin espacios en blanco
    hepatitis.columns = [col.rstrip() for col in hepatitis.columns]

    # Convertir niveles 1,2 a 0,1
    columns_to_map = ['Gender', 'Fever', 'Nausea/Vomting', 'Headache', 'Diarrhea', 
    'Fatigue & generalized bone ache', 'Jaundice', 'Epigastric pain']
    hepatitis[columns_to_map] = hepatitis[columns_to_map]-1
    hepatitis['Baselinehistological staging'] = hepatitis['Baselinehistological staging']-1

    with open('data/hepatitis_egypt.pkl', 'wb') as f:
        pickle.dump(hepatitis, f)
        
hepatitis

,Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,...,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,Baseline histological Grading,Baselinehistological staging
0,56,0,35,1,0,0,0,1,1,1,...,5,5,5,655330,634536,288194,5,5,13,1
1,46,0,29,0,1,1,0,1,1,0,...,57,123,44,40620,538635,637056,336804,31085,4,1
2,57,0,33,1,1,1,1,0,0,0,...,5,5,5,571148,661346,5,735945,558829,4,3
3,49,1,33,0,1,0,1,0,1,0,...,48,77,33,1041941,449939,585688,744463,582301,10,2
4,59,0,32,0,0,1,0,1,1,1,...,94,90,30,660410,738756,3731527,338946,242861,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,44,0,29,0,1,1,1,0,0,0,...,63,44,45,387795,55938,5,5,5,15,3
1381,55,0,34,0,1,1,0,0,0,0,...,97,64,41,481378,152961,393339,73574,236273,10,1
1382,42,0,26,1,1,0,0,0,1,0,...,87,39,24,612664,572756,806109,343719,160457,6,1
1383,52,0,29,1,0,0,1,1,1,0,...,48,81,43,139872,76161,515730,2460,696074,15,2


In [9]:
# Stratify train-val-test split
X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_split_stratify(df=hepatitis, target='Baselinehistological staging', SEED=1223)

# Standardization of continuous variables
X_train_scaled, X_val_scaled, X_test_scaled = utils.scale_numerical_variables(X_train, X_test, X_val, 
                                                                             numerical_variables= [
                                                                                "WBC", "RBC", "HGB", "Plat", 
                                                                                "AST 1", "ALT 1", "ALT4", "ALT 12", "ALT 24", "ALT 36", 
                                                                                "ALT 48", "ALT after 24 w", "RNA Base", "RNA 4", 
                                                                                "RNA 12", "RNA EOT", "RNA EF"
                                                                            ])

Class distribution in train:
3    217
2    213
0    202
1    199
Name: Baselinehistological staging, dtype: int64
Class distribution in val:
3    73
2    71
0    67
1    66
Name: Baselinehistological staging, dtype: int64
Class distribution in test:
3    72
2    71
1    67
0    67
Name: Baselinehistological staging, dtype: int64


* Noselection results

In [10]:
model1 = NoSelectionModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)       

model1.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model1.model.save('results/hepatitis_egypt/hepatitis_egypt_NoSelectionModel.h5')
with open('results/hepatitis_egypt/hepatitis_egypt_NoSelectionModel_history.pkl', 'wb') as f:
    pickle.dump(model1.history, f)

Epoch 1/50
26/26 [==============================] - 1s 7ms/step - loss: 1.9361 - sparse_categorical_accuracy: 0.2671 - val_loss: 1.3883 - val_sparse_categorical_accuracy: 0.2635
Epoch 2/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3885 - sparse_categorical_accuracy: 0.2611 - val_loss: 1.3868 - val_sparse_categorical_accuracy: 0.2635
Epoch 3/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3866 - sparse_categorical_accuracy: 0.2611 - val_loss: 1.3860 - val_sparse_categorical_accuracy: 0.2635
Epoch 4/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3862 - sparse_categorical_accuracy: 0.2611 - val_loss: 1.3856 - val_sparse_categorical_accuracy: 0.2635
Epoch 5/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3868 - sparse_categorical_accuracy: 0.2611 - val_loss: 1.3855 - val_sparse_categorical_accuracy: 0.2635
Epoch 6/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3857 - sparse_categorical_accuracy: 

In [11]:
model1.evaluate(X_test_scaled, y_test)

verbose1 = model1.get_verbose()

comparative_results.loc['NoSelection'] = [verbose1['results']['loss'], 
                                          verbose1['results']['accuracy'], 
                                          verbose1['results']['f1'], 
                                          verbose1['selected_features'].shape[0], 
                                          verbose1['selected_features']]
comparative_results

9/9 [==============================] - 0s 685us/step - loss: 1.3858 - sparse_categorical_accuracy: 0.2599


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.385829,0.259928,0.107248,28,"[Age, Gender, BMI, Fever, Nausea/Vomting, Head..."


* MonoFADL results

In [37]:
model2 = MonoFADLModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)

model2.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model2.model.save('results/hepatitis_egypt/hepatitis_egypt_MonoFADLModel.h5')
with open('results/hepatitis_egypt/hepatitis_egypt_MonoFADLModel_history.pkl', 'wb') as f:
    pickle.dump(model2.history, f)

Epoch 1/50
26/26 [==============================] - 1s 12ms/step - loss: 1.4539 - sparse_categorical_accuracy: 0.2395 - val_loss: 1.3443 - val_sparse_categorical_accuracy: 0.2383
Epoch 2/50
26/26 [==============================] - 0s 1ms/step - loss: 1.3188 - sparse_categorical_accuracy: 0.2274 - val_loss: 1.2877 - val_sparse_categorical_accuracy: 0.2563
Epoch 3/50
26/26 [==============================] - 0s 1ms/step - loss: 1.2590 - sparse_categorical_accuracy: 0.2407 - val_loss: 1.2267 - val_sparse_categorical_accuracy: 0.2635
Epoch 4/50
26/26 [==============================] - 0s 1ms/step - loss: 1.1974 - sparse_categorical_accuracy: 0.2587 - val_loss: 1.1641 - val_sparse_categorical_accuracy: 0.2563
Epoch 5/50
26/26 [==============================] - 0s 1ms/step - loss: 1.1349 - sparse_categorical_accuracy: 0.2359 - val_loss: 1.1001 - val_sparse_categorical_accuracy: 0.2635
Epoch 6/50
26/26 [==============================] - 0s 1ms/step - loss: 1.0692 - sparse_categorical_accuracy:

In [38]:
model2.evaluate(X_test_scaled, y_test)

verbose2 = model2.get_verbose()

comparative_results.loc['MonoFADL'] = [verbose2['results']['loss'], 
                                          verbose2['results']['accuracy'], 
                                          verbose2['results']['f1'], 
                                          verbose2['selected_features'].shape[0], 
                                          verbose2['selected_features']]
comparative_results

9/9 [==============================] - 0s 795us/step - loss: -2.0240 - sparse_categorical_accuracy: 0.2599


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.389501,0.263538,0.120655,28,"[Age , Gender, BMI, Fever, Nausea/Vomting, Hea..."
MonoFADL,-2.024016,0.259928,0.107248,0,[]


* MultiFADL One-versus-Rest results

In [39]:
model3 = MultiFADLModelOvR(
)

model3.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

for clas, model in model3.models.items():
    model.model.save(f'results/hepatitis_egypt/hepatitis_egypt_MultiFADLModel_{clas}.h5')    
    with open(f'results/hepatitis_egypt/hepatitis_egypt_MultiFADLModel_history_{clas}.pkl', 'wb') as f:
        pickle.dump(model.history, f)

--> Training model class 0 vs rest
Epoch 1/50
26/26 [==============================] - 1s 7ms/step - loss: 0.6179 - binary_accuracy: 0.6775 - val_loss: 0.5179 - val_binary_accuracy: 0.7581
Epoch 2/50
26/26 [==============================] - 0s 2ms/step - loss: 0.4868 - binary_accuracy: 0.7569 - val_loss: 0.4531 - val_binary_accuracy: 0.7581
Epoch 3/50
26/26 [==============================] - 0s 2ms/step - loss: 0.4276 - binary_accuracy: 0.7569 - val_loss: 0.3986 - val_binary_accuracy: 0.7581
Epoch 4/50
26/26 [==============================] - 0s 2ms/step - loss: 0.3761 - binary_accuracy: 0.7569 - val_loss: 0.3477 - val_binary_accuracy: 0.7581
Epoch 5/50
26/26 [==============================] - 0s 2ms/step - loss: 0.3255 - binary_accuracy: 0.7569 - val_loss: 0.2974 - val_binary_accuracy: 0.7581
Epoch 6/50
26/26 [==============================] - 0s 1ms/step - loss: 0.2755 - binary_accuracy: 0.7569 - val_loss: 0.2452 - val_binary_accuracy: 0.7581
Epoch 7/50
26/26 [=======================

In [40]:
model3.evaluate(X_test_scaled, y_test)

verbose3 = model3.get_verbose()

comparative_results.loc['MultiFADL'] = [verbose3['results']['loss'], 
                                          verbose3['results']['accuracy'], 
                                          verbose3['results']['f1'], 
                                          {clas: verbose3['selected_features'][clas].shape[0] for clas in verbose3['selected_features']}, 
                                          verbose3['selected_features']]

comparative_results

--> Evaluating model class 0 vs rest
9/9 [==============================] - 0s 686us/step - loss: -2.4831 - binary_accuracy: 0.7581
{'loss': -2.4830527305603027, 'accuracy': 0.7581227421760559, 'f1': 0.6538224894180089}
--> Evaluating model class 1 vs rest
9/9 [==============================] - 0s 654us/step - loss: -2.4323 - binary_accuracy: 0.7581
{'loss': -2.432304859161377, 'accuracy': 0.7581227421760559, 'f1': 0.6538224894180089}
--> Evaluating model class 2 vs rest
9/9 [==============================] - 0s 673us/step - loss: -2.7243 - binary_accuracy: 0.7437
{'loss': -2.724271774291992, 'accuracy': 0.743682324886322, 'f1': 0.6343625505452535}
--> Evaluating model class 3 vs rest
9/9 [==============================] - 0s 706us/step - loss: -2.8921 - binary_accuracy: 0.7401
{'loss': -2.892096996307373, 'accuracy': 0.7400721907615662, 'f1': 0.6295219976931259}


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.3895,0.263538,0.120655,28,"[Age , Gender, BMI, Fever, Nausea/Vomting, Hea..."
MonoFADL,-2.02402,0.259928,0.107248,0,[]
MultiFADL,"{0: -2.4830527305603027, 1: -2.432304859161377...","{0: 0.7581227421760559, 1: 0.7581227421760559,...","{0: 0.6538224894180089, 1: 0.6538224894180089,...","{0: 0, 1: 0, 2: 0, 3: 0, 'global': 0}","{0: [], 1: [], 2: [], 3: [], 'global': []}"


In [41]:
comparative_results.to_csv('results/hepatitis_egypt/hepatitis_egypt_ComparativeResults.csv')